In [1]:
using HTTP
using JSON

In [5]:
using CSV, DataFrames

In [17]:
using ProgressMeter

In [18]:
function google_distance_matrix_symmetric(locations::Array{String,1})
    if !haskey(ENV, "GAPI_KEY")
        error("Google API key not found.")
        return
    end
    key = ENV["GAPI_KEY"]

    locs = [foldl(replace, [", "=>"+", " "=>"+", ","=>"+"], init=s) for s in locations]

    N = length(locations)
    distancematrix = zeros(Int, N, N)
    @showprogress for i in 1:N
        for j in i+1:N
            s = locs[i]
            d = locs[j]
            url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins=$(s)&destinations=$(d)&key=$(key)"

            response = HTTP.request("GET", url)
            results = JSON.parse(String(response.body))["rows"][1]["elements"][1]

            if results["status"] != "OK"
                distancematrix[i,j] = -1
                distancematrix[j,i] = -1
                continue
            end

            distancematrix[i,j] = results["duration"]["value"]
            distancematrix[j,i] = results["duration"]["value"]
        end
    end

    return distancematrix
end;

In [27]:
state_list = CSV.read("../../data/geography/states.csv")

,state,abbrev,fips,capital_city,main_city,lat,long
,String,String,Int64,String,String,Float64,Float64
1,Alabama,AL,1,Montgomery,Birmingham,32.3182,-86.9023
2,Alaska,AK,2,Juneau,Anchorage,63.5888,-154.493
3,Arizona,AZ,4,Phoenix,Phoenix,34.0489,-111.094
4,Arkansas,AR,5,Little Rock,Little Rock,35.2011,-91.8318
5,California,CA,6,Sacramento,Los Angeles,36.7783,-119.418
6,Colorado,CO,8,Denver,Denver,39.5501,-105.782
7,Connecticut,CT,9,Hartford,Bridgeport,41.6032,-73.0877
8,Delaware,DE,10,Dover,Wilmington,38.9108,-75.5277
9,District of Columbia,DC,11,Washington DC,Washington,38.906,-77.0334


In [19]:
d1 = google_distance_matrix_symmetric(state_list.state[:])

Progress: 100%|█████████████████████████████████████████| Time: 0:02:26


52×52 Array{Int64,2}:
      0  255479   89850   26749  116722  …  140757   40475   51193   90341
 255479       0  220437  232522  205527     156716  246502  207165  177585
  89850  220437       0   69786   45071      85379  109183   96871   58301
  26749  232522   69786       0   97032     118153   44203   40289   67736
 116722  205527   45071   97032       0      57238  136161  116286   63471
  80541  196545   44583   57186   64475  …   75378   81836   60089   22556
  62969  260741  134005   72140  160247     153705   33325   57050  108375
  50547  256640  126982   60362  153961     149810   23341   53154  104479
  45186  252279  122181   54759  149159     145338   18870   48683  100008
  31818  277222  117131   54379  145405     164188   50482   74300  113771
  16528  259093  103673   36251  130651  …  146301   35224   56413   95885
     -1      -1      -1      -1      -1         -1      -1      -1      -1
 120629  177159   62084   97440   52119      36895  120897   90327   33962
   

In [25]:
dist_matrix_1 = DataFrame(d1)
rename!(dist_matrix_1, state_list.abbrev[:])

,AL,AK,AZ,AR,CA,CO,CT,DE,DC,FL
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0,255479,89850,26749,116722,80541,62969,50547,45186,31818
2,255479,0,220437,232522,205527,196545,260741,256640,252279,277222
3,89850,220437,0,69786,45071,44583,134005,126982,122181,117131
4,26749,232522,69786,0,97032,57186,72140,60362,54759,54379
5,116722,205527,45071,97032,0,64475,160247,153961,149159,145405
6,80541,196545,44583,57186,64475,0,104051,99239,94878,104944
7,62969,260741,134005,72140,160247,104051,0,16138,18898,66808
8,50547,256640,126982,60362,153961,99239,16138,0,6375,54277
9,45186,252279,122181,54759,149159,94878,18898,6375,0,48947


In [26]:
CSV.write("state_dist_google.csv", dist_matrix_1);

In [30]:
city_list = map(s -> s.main_city * " " * s.abbrev, eachrow(state_list))

52-element Array{String,1}:
 "Birmingham AL"
 "Anchorage AK"
 "Phoenix AZ"
 "Little Rock AR"
 "Los Angeles CA"
 "Denver CO"
 "Bridgeport CT"
 "Wilmington DE"
 "Washington DC"
 "Jacksonville FL"
 "Atlanta GA"
 "Honolulu HI"
 "Boise ID"
 ⋮
 "Providence RI"
 "Charleston SC"
 "Sioux Falls SD"
 "Nashville TN"
 "Houston TX"
 "Salt Lake City UT"
 "Burlington VT"
 "Virginia Beach VA"
 "Seattle WA"
 "Charleston WV"
 "Milwaukee WI"
 "Cheyenne WY"

In [31]:
d2 = google_distance_matrix_symmetric(city_list)

Progress: 100%|█████████████████████████████████████████| Time: 0:02:15


52×52 Array{Int64,2}:
      0  251643   88168   20264  107594  …  137293   30569   40370   72784
 251643       0  218198  235143  218215     155146  243438  212150  188749
  88168  218198       0   69185   20471      78587  104263   97545   51227
  20264  235143   69185       0   88129     120504   39128   39255   54937
 107594  218215   20471   88129       0      63480  122790  108635   58761
  69707  194138   46334   50644   54551  …   71474   71027   55121    5907
  55665  262361  133770   70174  151562     155784   33157   51770   96361
  45767  257983  126711   60276  145239     151400   23937   47386   91977
  40071  254932  122978   54580  141506     148175   20400   44162   88753
  25764  271298  105427   45624  125160     158555   34743   61309   94046
   8240  253739   95863   28099  115025  …  140896   28007   43650   76388
     -1      -1      -1      -1      -1         -1      -1      -1      -1
 111423  181631   51245   93162   47990      28097  112154   91481   39046
   

In [33]:
dist_matrix_2 = DataFrame(d2)
col_names = map(s -> s.main_city * "." * s.abbrev, eachrow(state_list))
rename!(dist_matrix_2, col_names)

,Birmingham.AL,Anchorage.AK,Phoenix.AZ,Little Rock.AR,Los Angeles.CA,Denver.CO
,Int64,Int64,Int64,Int64,Int64,Int64
1,0,251643,88168,20264,107594,69707
2,251643,0,218198,235143,218215,194138
3,88168,218198,0,69185,20471,46334
4,20264,235143,69185,0,88129,50644
5,107594,218215,20471,88129,0,54551
6,69707,194138,46334,50644,54551,0
7,55665,262361,133770,70174,151562,98048
8,45767,257983,126711,60276,145239,93078
9,40071,254932,122978,54580,141506,89786


In [34]:
CSV.write("state_cities_dist_google.csv", dist_matrix_2);